## M1 monetary aggregates

In [4]:
import pandas as pd

# Load the CSV files
m1_eu = pd.read_csv(r'C:\Users\luish\OneDrive - University of Leeds\9. DISERTATION\2. Development\code\1. data\M1_EU.csv', parse_dates=True)
m1_usa = pd.read_csv(r'C:\Users\luish\OneDrive - University of Leeds\9. DISERTATION\2. Development\code\1. data\M1_USA.csv', parse_dates=True)

# Select only the necessary columns
m1_eu = m1_eu.iloc[:, [0, 2]]
m1_usa = m1_usa.iloc[:, :2]
m1_eu.columns = ['date', 'M1']
m1_usa.columns = ['date', 'M1']

# Convert 'date' column to datetime
m1_eu['date'] = pd.to_datetime(m1_eu['date'])
m1_usa['date'] = pd.to_datetime(m1_usa['date'])

# Step 1: Filter from 2014-12-31 to 2024-12-31 to allow interpolation
raw_start, final_start, end = '2014-12-31', '2015-01-01', '2024-12-31'
m1_eu = m1_eu[(m1_eu['date'] >= raw_start) & (m1_eu['date'] <= end)]
m1_usa = m1_usa[(m1_usa['date'] >= raw_start) & (m1_usa['date'] <= end)]

# Step 2: Set date index for resampling
m1_eu.set_index('date', inplace=True)
m1_usa.set_index('date', inplace=True)

# Step 3: Create full daily date index
full_daily_index = pd.date_range(start=raw_start, end=end, freq='D')

# Step 4: Reindex and interpolate
m1_eu_daily = m1_eu.reindex(full_daily_index).interpolate(method='linear')
m1_usa_daily = m1_usa.reindex(full_daily_index).interpolate(method='linear')

# Step 5: Trim from 2015-01-01 onwards
m1_eu_daily = m1_eu_daily.loc[final_start:]
m1_usa_daily = m1_usa_daily.loc[final_start:]

# Reset index and rename
m1_eu_daily.index.name = 'date'
m1_usa_daily.index.name = 'date'
m1_eu_daily.reset_index(inplace=True)
m1_usa_daily.reset_index(inplace=True)

# Step 6: Save final interpolated files
m1_eu_daily.to_csv(r'C:\Users\luish\OneDrive - University of Leeds\9. DISERTATION\2. Development\code\2. preprocesing\M1_EU_daily.csv', index=False)
m1_usa_daily.to_csv(r'C:\Users\luish\OneDrive - University of Leeds\9. DISERTATION\2. Development\code\2. preprocesing\M1_USA_daily.csv', index=False)

print("Interpolated daily series saved successfully from 2015-01-01.")




Interpolated daily series saved successfully from 2015-01-01.


## Interest rate - 3 months

In [5]:
import pandas as pd

# Load interest rate files
r3m_eu = pd.read_csv(
    r'C:\Users\luish\OneDrive - University of Leeds\9. DISERTATION\2. Development\code\1. data\interest_rate_3m_EU.csv',
    parse_dates=True
)
r3m_usa = pd.read_csv(
    r'C:\Users\luish\OneDrive - University of Leeds\9. DISERTATION\2. Development\code\1. data\interest_rate_3m_USA.csv',
    parse_dates=True
)

# Show original column names
print("EU columns:", r3m_eu.columns)
print("USA columns:", r3m_usa.columns)

# Keep only the first two columns and rename them
r3m_eu = r3m_eu.iloc[:, :2]
r3m_usa = r3m_usa.iloc[:, :2]
r3m_eu.columns = ['date', 'rate']
r3m_usa.columns = ['date', 'rate']

# Convert 'date' to datetime format
r3m_eu['date'] = pd.to_datetime(r3m_eu['date'])
r3m_usa['date'] = pd.to_datetime(r3m_usa['date'])

# Step 1: Keep a slightly wider window to ensure fill coverage
raw_start, final_start, end = '2014-12-31', '2015-01-01', '2024-12-31'
r3m_eu = r3m_eu[(r3m_eu['date'] >= raw_start) & (r3m_eu['date'] <= end)]
r3m_usa = r3m_usa[(r3m_usa['date'] >= raw_start) & (r3m_usa['date'] <= end)]

# Step 2: Set date as index
r3m_eu.set_index('date', inplace=True)
r3m_usa.set_index('date', inplace=True)

# Step 3: Create full daily date index
daily_index = pd.date_range(start=raw_start, end=end, freq='D')

# Step 4: Reindex and forward-fill (step function behavior)
r3m_eu_daily = r3m_eu.reindex(daily_index).ffill()
r3m_usa_daily = r3m_usa.reindex(daily_index).ffill()

# Step 5: Trim to start from 2015-01-01
r3m_eu_daily = r3m_eu_daily.loc[final_start:]
r3m_usa_daily = r3m_usa_daily.loc[final_start:]

# Step 6: Reset index and rename
r3m_eu_daily.index.name = 'date'
r3m_usa_daily.index.name = 'date'
r3m_eu_daily.reset_index(inplace=True)
r3m_usa_daily.reset_index(inplace=True)

# Step 7: Save daily resampled datasets
r3m_eu_daily.to_csv(
    r'C:\Users\luish\OneDrive - University of Leeds\9. DISERTATION\2. Development\code\2. preprocesing\interest_rate_3m_EU_daily.csv',
    index=False
)
r3m_usa_daily.to_csv(
    r'C:\Users\luish\OneDrive - University of Leeds\9. DISERTATION\2. Development\code\2. preprocesing\interest_rate_3m_USA_daily.csv',
    index=False
)

print("Daily interest rate files saved with forward-fill applied from 2015-01-01.")
                                          

EU columns: Index(['observation_date', 'IR3TIB01EZM156N'], dtype='object')
USA columns: Index(['observation_date', 'TB3MS'], dtype='object')
Daily interest rate files saved with forward-fill applied from 2015-01-01.


## Industrial production

In [6]:
import pandas as pd
import numpy as np

# ----------------------------------------
# Load Eurozone Industrial Production (Quarterly)
# ----------------------------------------

eu_file = r'C:\Users\luish\OneDrive - University of Leeds\9. DISERTATION\2. Development\code\1. data\INDPRO_Q_EU.csv'
eu_ip = pd.read_csv(eu_file, parse_dates=['observation_date'])
eu_ip.columns = ['date', 'IP_EU']
eu_ip.set_index('date', inplace=True)

# Reindex to monthly start-of-month dates using forward-fill
monthly_index = pd.date_range(start='2014-12-31', end='2024-12-31', freq='MS')
eu_ip_monthly = eu_ip.reindex(monthly_index, method='ffill')

# Then reindex to daily and interpolate linearly
daily_index = pd.date_range(start='2015-01-01', end='2024-12-31', freq='D')
eu_ip_daily = eu_ip_monthly.reindex(daily_index).interpolate(method='linear')
eu_ip_daily.index.name = 'date'
eu_ip_daily.reset_index(inplace=True)

# ----------------------------------------
# Load US Industrial Production (Monthly)
# ----------------------------------------

us_file = r'C:\Users\luish\OneDrive - University of Leeds\9. DISERTATION\2. Development\code\1. data\INDPRO_USA.csv'
us_ip = pd.read_csv(us_file, parse_dates=['observation_date'])
us_ip.columns = ['date', 'IP_USA']
us_ip.set_index('date', inplace=True)

# Reindex to daily and interpolate linearly
us_ip_daily = us_ip.reindex(daily_index).interpolate(method='linear')
us_ip_daily.index.name = 'date'
us_ip_daily.reset_index(inplace=True)

# ----------------------------------------
# Save both as daily CSVs
# ----------------------------------------

eu_ip_daily.to_csv(
    r'C:\Users\luish\OneDrive - University of Leeds\9. DISERTATION\2. Development\code\2. preprocesing\INDPRO_EU_daily.csv',
    index=False
)

us_ip_daily.to_csv(
    r'C:\Users\luish\OneDrive - University of Leeds\9. DISERTATION\2. Development\code\2. preprocesing\INDPRO_USA_daily.csv',
    index=False
)

print("Daily production series saved successfully for EU and USA.")


Daily production series saved successfully for EU and USA.
